## TensorFlow 2.0: Functions, not Sessions.
[TensorFlow 2.0: Functions, not Sessions.](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md)

In [3]:
import tensorflow as tf

### 目标

在 2.0 中让 TensorFlow 更加的 Python 化。所以有下面 5 个提议：

- 提倡用 Python 函数来作为对图计算的封装。（即当函数被调用时，图就被执行，而不是通过 Session 来创建）。


In [4]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: id=65, shape=(3, 3), dtype=float32, numpy=
array([[1.0146681 , 0.3902194 , 1.0057268 ],
       [1.3517941 , 0.43906617, 1.164418  ],
       [0.9367994 , 0.22281522, 0.7040087 ]], dtype=float32)>

当一个函数被 `tf.function` 标注后，你可像调用其它函数一样调用它。但是当在 GPU 或 TPU 上运行时，函数会被编译成 graph， 这样会获得更大的性能。

In [5]:
simple_nn_layer

**注意**，你不必为所有的函数都进行标注，在一个标注的函数调用了另一个函数，另一个函数会被自动当作图来计算。

In [6]:
def linear_layer(x):
    return 2 * x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: id=79, shape=(3,), dtype=int32, numpy=array([3, 5, 7], dtype=int32)>

对于一不复杂简单的 ops, 函数可能会更快些。但是对于一些复杂计算量大的操作，如（convolutions),可能就不会这么快了。

In [7]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 200])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

Eager conv: 1.325879008000129
Function conv: 1.3980411940001432
Note how there's not much difference in performance for convolutions


In [8]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2
# warm up
lstm_cell(input, state); lstm_fn(input, state)
print("eager lstm:", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("function lstm:", timeit.timeit(lambda: lstm_fn(input, state), number=10))

eager lstm: 0.027461488999961148
function lstm: 0.003729850999889095


### Use Python control flow

AutoGraph 会将他们转换成特定年 TensorFlow, 如，`if` 语句会被转换成 `tf.cond()`.